In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import json
import ast

In [2]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [3]:
from statsmodels.stats import diagnostic
import statsmodels.formula.api as smf
import statsmodels.api as sm

/home/leo/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/leo/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [4]:
df = pd.read_csv('./data/movie_features.csv')

In [5]:
df.head()

,movie_id,title,budget,revenue,release_date,runtime,original_language,genres,number_characters,below18,above60,male_ratio,average_actor_exp,multiplier,year,year_group,category
0,951491.0,Saw X,13000000.0,100900000.0,2023-09-26,118.0,en,"['Horror', 'Thriller']",7.0,0.000,0.142857,0.571429,5.857143,7.761538,2023,2020,low
1,678512.0,Sound of Freedom,14500000.0,247800000.0,2023-07-03,131.0,en,"['Action', 'Drama']",7.0,0.000,0.142857,0.857143,7.285714,17.089655,2023,2020,low
2,820525.0,After Everything,14000000.0,9700000.0,2023-09-13,93.0,en,"['Romance', 'Drama']",10.0,0.000,0.000000,0.500000,3.200000,0.692857,2023,2020,low
3,1008042.0,Talk to Me,4500000.0,90600000.0,2023-07-26,95.0,en,"['Horror', 'Thriller']",8.0,0.125,0.125000,0.500000,1.250000,20.133333,2023,2020,low
4,1058694.0,Radical,10000000.0,16100000.0,2023-10-19,127.0,en,['Drama'],1.0,0.000,1.000000,1.000000,9.000000,1.610000,2023,2020,low


In [6]:
df['normalized_runtime'] = (df['runtime'] - df['runtime'].mean())/df['runtime'].std()

In [7]:
mod_1 = smf.ols(formula='multiplier ~ normalized_runtime', data=df)

In [8]:
np.random.seed(2)
res = mod_1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             multiplier   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     4.489
Date:                Thu, 14 Dec 2023   Prob (F-statistic):             0.0343
Time:                        13:30:58   Log-Likelihood:                -7046.8
No. Observations:                1701   AIC:                         1.410e+04
Df Residuals:                    1699   BIC:                         1.411e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.2151      0

In [9]:
# Performing one-hot encoding for the 'track' column
original_language_onehot = pd.get_dummies(df['original_language'], prefix='onehot', prefix_sep='_')

# Concatenate the new one-hot encoded columns to the original dataframe
df = pd.concat([df, original_language_onehot], axis=1)

for col in df.columns:
    print(col)

movie_id
title
budget
revenue
release_date
runtime
original_language
genres
number_characters
below18
above60
male_ratio
average_actor_exp
multiplier
year
year_group
category
normalized_runtime
onehot_cn
onehot_cs
onehot_da
onehot_de
onehot_en
onehot_es
onehot_fi
onehot_fr
onehot_hi
onehot_id
onehot_it
onehot_ja
onehot_kn
onehot_ko
onehot_la
onehot_no
onehot_pt
onehot_ru
onehot_sv
onehot_te
onehot_th
onehot_tr
onehot_zh


In [10]:
# Select all the onehot feature names previously created
# selected_cols = [col for col in df.columns if 'onehot' in col]

grouped_by_language_df = df.groupby(['original_language']).size()
print(grouped_by_language_df)

original_language
cn      10
cs       1
da       5
de       4
en    1509
es      21
fi       1
fr      34
hi      24
id       2
it       5
ja      22
kn       1
ko      20
la       1
no       4
pt       1
ru      11
sv       3
te       2
th       2
tr       1
zh      17
dtype: int64


In [11]:
# Selecting only languages above a certain threshold
mod_2 = sm.OLS(df['multiplier'], df[['onehot_cn', 'onehot_en', 'onehot_es', 'onehot_fr', 'onehot_hi', 'onehot_ja', 'onehot_ko', 'onehot_ru', 'onehot_zh']])

In [12]:
np.random.seed(2)
res = mod_2.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.138
Model:                            OLS   Adj. R-squared (uncentered):              0.134
Method:                 Least Squares   F-statistic:                              30.19
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    2.92e-49
Time:                        13:31:03   Log-Likelihood:                         -7016.3
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1692   BIC:                                  1.410e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [13]:
# Trying to improve it by removing non-statistically significant ones
mod_3 = sm.OLS(df['multiplier'], df[['onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])

In [14]:
np.random.seed(2)
res = mod_3.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.133
Model:                            OLS   Adj. R-squared (uncentered):              0.131
Method:                 Least Squares   F-statistic:                              64.95
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    3.66e-51
Time:                        13:31:06   Log-Likelihood:                         -7021.9
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1697   BIC:                                  1.407e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [15]:
# Trying to combine with normalized runtime
mod_4 = sm.OLS(df['multiplier'], df[['normalized_runtime', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_4.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.134
Model:                            OLS   Adj. R-squared (uncentered):              0.131
Method:                 Least Squares   F-statistic:                              52.36
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    1.25e-50
Time:                        13:31:08   Log-Likelihood:                         -7020.9
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1696   BIC:                                  1.408e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [16]:
# Non sense to add runtime, trying number of characters
mod_5 = sm.OLS(df['multiplier'], df[['number_characters', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_5.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.135
Model:                            OLS   Adj. R-squared (uncentered):              0.133
Method:                 Least Squares   F-statistic:                              53.04
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    2.90e-51
Time:                        13:31:10   Log-Likelihood:                         -7019.4
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1696   BIC:                                  1.408e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [17]:
# Non sense to add runtime, trying to add other features
mod_6 = sm.OLS(df['multiplier'], df[['below18', 'above60', 'male_ratio', 'average_actor_exp', 'number_characters', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_6.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.139
Model:                            OLS   Adj. R-squared (uncentered):              0.134
Method:                 Least Squares   F-statistic:                              30.35
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    1.61e-49
Time:                        13:31:13   Log-Likelihood:                         -7015.7
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1692   BIC:                                  1.410e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [18]:
# Non sense to add runtime, trying to add other features
mod_7 = sm.OLS(df['multiplier'], df[['average_actor_exp', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_7.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.138
Model:                            OLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              54.25
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    2.20e-52
Time:                        13:31:18   Log-Likelihood:                         -7016.8
No. Observations:                1701   AIC:                                  1.404e+04
Df Residuals:                    1696   BIC:                                  1.407e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [19]:
df['genres'] = df['genres'].apply(ast.literal_eval)

# Extract unique genres
unique_genres = set(genre for genres_list in df['genres'] for genre in genres_list)

# Create one-hot encoded columns for each genre
for genre in unique_genres:
    df['onehot_genre_' + genre.lower()] = df['genres'].apply(lambda x: 1 if genre in x else 0)

In [20]:
# Select all the onehot feature from genres
selected_cols = [col for col in df.columns if 'onehot_genre_' in col]

In [21]:
# Non sense to add runtime, trying to add other features
mod_7 = sm.OLS(df['multiplier'], df[selected_cols + ['average_actor_exp', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_7.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.163
Model:                            OLS   Adj. R-squared (uncentered):              0.151
Method:                 Least Squares   F-statistic:                              13.65
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    1.01e-49
Time:                        13:31:52   Log-Likelihood:                         -6991.3
No. Observations:                1701   AIC:                                  1.403e+04
Df Residuals:                    1677   BIC:                                  1.416e+04
Df Model:                          24                                                  
Covariance Type:            nonrobust                                                  
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

In [22]:
for col in selected_cols:
    print('{} has {} values'.format(col, df[col].sum()))

onehot_genre_horror has 267 values
onehot_genre_science fiction has 302 values
onehot_genre_adventure has 508 values
onehot_genre_thriller has 473 values
onehot_genre_action has 584 values
onehot_genre_war has 51 values
onehot_genre_western has 16 values
onehot_genre_drama has 619 values
onehot_genre_romance has 237 values
onehot_genre_fantasy has 277 values
onehot_genre_mystery has 144 values
onehot_genre_comedy has 538 values
onehot_genre_history has 73 values
onehot_genre_music has 39 values
onehot_genre_crime has 232 values
onehot_genre_family has 253 values
onehot_genre_documentary has 6 values
onehot_genre_animation has 172 values
onehot_genre_tv movie has 1 values


In [23]:
# Selected most populated ones
selected_cols.remove('onehot_genre_war')
selected_cols.remove('onehot_genre_documentary')
selected_cols.remove('onehot_genre_music')
selected_cols.remove('onehot_genre_western')
selected_cols.remove('onehot_genre_tv movie')
selected_cols.remove('onehot_genre_history')

mod_8 = sm.OLS(df['multiplier'], df[selected_cols + ['average_actor_exp', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_8.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.163
Model:                            OLS   Adj. R-squared (uncentered):              0.154
Method:                 Least Squares   F-statistic:                              18.16
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    4.51e-53
Time:                        13:32:44   Log-Likelihood:                         -6992.1
No. Observations:                1701   AIC:                                  1.402e+04
Df Residuals:                    1683   BIC:                                  1.412e+04
Df Model:                          18                                                  
Covariance Type:            nonrobust                                                  
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

## Based on the previous analysis, create the best model for low/high budget

In [24]:
# Trying to create two models based on the multiplier (low/high)
df_low = df[df['category'] == 'low']
df_high = df[df['category'] == 'high']

In [37]:
mod_9 = sm.OLS(df_low['multiplier'], df_low[selected_cols + ['average_actor_exp', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_9.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.166
Model:                            OLS   Adj. R-squared (uncentered):              0.148
Method:                 Least Squares   F-statistic:                              9.244
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                    1.78e-23
Time:                        13:50:28   Log-Likelihood:                         -3793.6
No. Observations:                 855   AIC:                                      7623.
Df Residuals:                     837   BIC:                                      7709.
Df Model:                          18                                                  
Covariance Type:            nonrobust                                                  
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

In [38]:
# Remove some languages since not represented in high budget dataset
mod_10 = sm.OLS(df_high['multiplier'], df_high[selected_cols + ['average_actor_exp', 'number_characters', 'onehot_en']])
np.random.seed(2)
res = mod_10.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.614
Model:                            OLS   Adj. R-squared (uncentered):              0.606
Method:                 Least Squares   F-statistic:                              82.44
Date:                Thu, 14 Dec 2023   Prob (F-statistic):                   5.12e-159
Time:                        13:51:10   Log-Likelihood:                         -1974.3
No. Observations:                 846   AIC:                                      3981.
Df Residuals:                     830   BIC:                                      4056.
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

## Using Decision Tree Regressor

In [56]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

In [57]:
# Helper function returning a numpy array with given dataframe and columns
def numpy_helper(df, cols):
    # Selecting the specified columns from the dataframe
    selected_data = df[cols]

    # Converting the selected data to a numpy array and cast as float
    numpy_array = selected_data.to_numpy(dtype=float)
    return numpy_array

In [87]:
X = numpy_helper(df, [col for col in df.columns if 'onehot' in col] + ['number_characters', 'below18', 'above60', 'male_ratio', 'average_actor_exp'])
y = numpy_helper(df, ['multiplier'])

print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (1701, 18)
Shape of y: (1701, 1)


In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [60]:
# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train.ravel())

# Predict on the test set
y_pred = rf_regressor.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 82.38371976164535


In [61]:
# Assuming y_true are the actual target values, and y_pred are the predicted values
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: -0.08734614711761335


In [112]:
# Assuming X and y are your feature matrix and target variable
decision_tree_model = DecisionTreeRegressor()
decision_tree_model.fit(X_train, y_train.ravel())

# Predict on the test set
y_pred = decision_tree_model.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 227.32138958830836


In [54]:
# Assuming y_true are the actual target values, and y_pred are the predicted values
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: -0.7939450966422106


## Same idea but for low/high budget datasets

### High budget

In [102]:
X = numpy_helper(df_high, [col for col in df.columns if 'onehot' in col] + ['number_characters', 'average_actor_exp'])
y = numpy_helper(df_high, ['multiplier'])

print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (846, 44)
Shape of y: (846, 1)


In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [104]:
# Initialize the Random Forest Regressor
rf_regressor_2 = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)

# Train the model
rf_regressor_2.fit(X_train, y_train.ravel())

# Predict on the test set
y_pred = rf_regressor_2.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 10.490267031409024


In [105]:
# Assuming y_true are the actual target values, and y_pred are the predicted values
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: 0.02173571562971599


In [106]:
# Assuming X and y are your feature matrix and target variable
decision_tree_model = DecisionTreeRegressor()
decision_tree_model.fit(X_train, y_train.ravel())

# Predict on the test set
y_pred = decision_tree_model.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 15.54091442514514


In [107]:
# Assuming y_true are the actual target values, and y_pred are the predicted values
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: -0.44925972647355894


### Low budget

In [108]:
X = numpy_helper(df_low, [col for col in df.columns if 'onehot' in col] + ['number_characters', 'average_actor_exp'])
y = numpy_helper(df_low, ['multiplier'])

print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (855, 44)
Shape of y: (855, 1)


In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [110]:
# Initialize the Random Forest Regressor
rf_regressor_3 = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)

# Train the model
rf_regressor_3.fit(X_train, y_train.ravel())

# Predict on the test set
y_pred = rf_regressor_3.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 170.7076721161348


In [111]:
# Assuming y_true are the actual target values, and y_pred are the predicted values
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: 0.054805000800541404


In [113]:
# Assuming X and y are your feature matrix and target variable
decision_tree_model = DecisionTreeRegressor()
decision_tree_model.fit(X_train, y_train.ravel())

# Predict on the test set
y_pred = decision_tree_model.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 258.9167696779585


In [114]:
# Assuming y_true are the actual target values, and y_pred are the predicted values
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: -0.43360185792934525
